This Notebook will be used to refactor my av_fcff.py module to a more correct form


In [1]:
    """This Python Module calculates a value for a common stock.  It does this by calculating an Enterprise Value , summing the present values of free cash flow to firm estimats plus a estimate of a termnal value.  To geneeral a pershare value it adds back cash and equivalents and subtracts debt.  The adjusted value is then divided by the number of shares outstanding to arrive at a per share value.
    """

'This Python Module calculates a value for a common stock.  It does this by calculating an Enterprise Value , summing the present values of free cash flow to firm estimats plus a estimate of a termnal value.  To geneeral a pershare value it adds back cash and equivalents and subtracts debt.  The adjusted value is then divided by the number of shares outstanding to arrive at a per share value.\n'

In [2]:
from dataclasses import dataclass
from typing import Optional
from datetime import date
import csv
import hg_dcflib


## Define the constants used in the module

In [3]:
EQ_PREM = 0.0441    # Damodaran 05/01/2025
MARGINAL_TAX_RATE = 0.26
STABLE_BETA = 1.0
COMPANY = input("Input company ticker: ").upper()
GROWTH_PERIOD = int(input("Input growth period: "))
INDUSTRY = hg_dcflib.get_industry(COMPANY)
with open("/Users/jhess/Development/Alpha2/data/ApiKey.txt") as f:
        MY_API_KEY = f.readline()
with open("/Users/jhess/Development/Alpha2/data/fred_api.txt") as f:
        FRED_KEY = f.readline()
RD_YEARS = hg_dcflib.get_rAndD_years(INDUSTRY) + 1
UNLEVERED_BETA = hg_dcflib.get_beta(INDUSTRY)
RISK_FREE = hg_dcflib.get_risk_free(FRED_KEY)      



Industry Group Metals & Mining
Years = 5
Beta 1.1279972137776122
0.042800000000000005


## Class for Valuation

In [ ]:
@dataclass
class Stock_Value:
    
    valuation_date: str
    ticker: str
    industry: str
    beta: float
    price: float
    shares_outstanding: float
    risk_free_rate: float
    eq_premium: float
    #
    #calcukated values in dataclass methods
    growth_rate: float = None
    fcff_value: float = None
    terminal_value: float = None
    ent_value: float = None
    share_value: float = None
    margin_of_safety: float = None
    
   
   
    
    
    
    
    

## Functions


In [ ]:
def income_statement(COMPANY, MY_API_KEY):
    inc_stmnt = hg_dcflib.get_inc_stmnt(COMPANY, MY_API_KEY)
    # with open(f"data/{COMPANY}inc_stmnt.csv", "w", newline="") as f:
    #     w = csv.DictWriter(f, inc_stmnt.keys())
    #     w.writeheader()
    #     w.writerow(inc_stmnt)
        
    return inc_stmnt

def balance_sheet(COMPANY, MY_API_KEY):
    bal_sht = hg_dcflib.get_bal_sheet(COMPANY, MY_API_KEY)
    # with open(f"data/{COMPANY}bal_Sht.csv", "w", newline="") as f:
    #     w = csv.DictWriter(f, bal_Sht.keys())
    #     w.writeheader()
    #     w.writerow(bal_Sht)
    return bal_sht

def cash_flow_statement(COMPANY, MY_API_KEY):
    cash_flw = hg_dcflib.get_cash_flow(COMPANY, MY_API_KEY)
    # with open(f"data/{COMPANY}cashFlw.csv", "w") as f:
    #     w = csv.DictWriter(f, cash_flw.keys())
    #     w.writeheader()
    #     w.writerow(cash_flw)
    return cash_flw

def enterprise_quote(COMPANY, MY_API_KEY):
    ent_quote = hg_dcflib.get_quote(COMPANY, MY_API_KEY)
    return ent_quote

def calc_capital_expenditures(cash_flw):
    capex = (cash_flw["capex"][0] + cash_flw["capex"][1] + cash_flw["capex"]
             [2] + cash_flw["capex"][3] + cash_flw["capex"][4]) / 5
    return capex
    
def calc_chng_wc(bal_sht):
    curr_yr_nc_wc =  (bal_sht["total_current_assets"][0] - bal_sht["cash_and_equivalents"][0]
    ) - (bal_sht["total_current_liabilities"][0] - bal_sht["short_term_debt"][0] - bal_sht["current_long_debt"][0])
    prior_yr_nc_wc = (
        bal_sht["total_current_assets"][1] - bal_sht["cash_and_equivalents"][1]
    ) - (bal_sht["total_current_liabilities"][1] - bal_sht["short_term_debt"][1] - bal_sht["current_long_debt"][1])
    chng_nc_wc = curr_yr_nc_wc - prior_yr_nc_wc
    return chng_nc_wc


def capitalizerAndD(COMPANY, RD_YEARS, MY_API_KEY):
    rdTable = hg_dcflib.get_rAndD(COMPANY, RD_YEARS, MY_API_KEY)
    rd_table = {}
    rd_expense = []
    unamort_percent = []
    unamort_amt = []
    curr_year_amortization = []
    amort_percentage = 1.0 / RD_YEARS
    rd_asset_value = 0
    rd_amort_amt = 0
    for year in range(RD_YEARS):
        # print(year, rdTable['researchAndDevelopment'][year])
        rd_expense.append(rdTable["research_and_development"][year])
        unamort_percent.append(1.0 - (1.0 / RD_YEARS * year))
        unamort_amt.append(rd_expense[year] * unamort_percent[year])
        if year == 0:
            curr_year_amortization.append(0.00)
        else:
            curr_year_amortization.append(
                rd_expense[year] * amort_percentage)

        rd_asset_value += unamort_amt[year]
        rd_amort_amt += curr_year_amortization[year]
    rd_table["rAndDExpense"] = rd_expense
    rd_table["unamortized_percent"] = unamort_percent
    rd_table["unamort_amount"] = unamort_amt
    rd_table["amort_amt"] = curr_year_amortization
    rd_table["RD_Asset_Value"] = rd_asset_value
    rd_table["RD_Amortization_Amt"] = rd_amort_amt

    return rd_table

def calc_fcff(inc_stmnt, bal_sht, cash_flw):
    ebiat = inc_stmnt['operating_income'][0] - inc_stmnt['income_tax_expense'][0]
    print(f"ebiat {ebiat}")
    capex = calc_capital_expenditures(cash_flw)
    print(f"Capex {capex}")
    chng_nc_wc = calc_chng_wc(bal_sht)
    print(f"Change WC {chng_nc_wc}")
    depreciation = cash_flw['depreciation'][0]
    print(f"Depreciation {depreciation}")
    fcff = ebiat - capex + depreciation + chng_nc_wc
    print(f"FCFF {fcff}")
    fcff_data = [ebiat, capex, chng_nc_wc, depreciation, fcff]
    return fcff_data
    
def calc_reinvestment(ebiat, capex, depreciation, chng_nc_wc, amort_schedule):
    firm_reinvestment = capex - depreciation + chng_nc_wc + \
        amort_schedule["rAndDExpense"][0] - \
        amort_schedule["RD_Amortization_Amt"]
    print(f"Firm Reinvestment {firm_reinvestment}")
    return firm_reinvestment

def calc_adj_ebiat(ebiat, amort_schedule):
    adjusted_ebiat = ebiat + amort_schedule["rAndDExpense"][0] - amort_schedule["RD_Amortization_Amt"]
    print(f"Adjusted ebiat {adjusted_ebiat}")
    return adjusted_ebiat

def calc_tax_rate(inc_stmnt):
    eff_tax_rate = inc_stmnt["operating_income"][0] / inc_stmnt["income_tax_expense"][0]
    return eff_tax_rate

def calc_return_on_capital(adjusted_ebiat, reinvestment_rate):
    return_on_capital = adjusted_ebiat / reinvestment_rate
    return return_on_capital

 def calc_growth_rate(reinvestment_rate, return_on_capital):
    growth_rate = reinvestment_rate * return_on_capital 
    
    return growth_rate

def calc_pv_fcff( curr_yr_fcff, growth_rate, GROWTH_PERIOD):
    fcfff_table = []
    pv_fcff = 0
    
    


def main():
    inc_stmnt = income_statement(COMPANY, MY_API_KEY)
    print(f"Inc Stmnt {inc_stmnt}")
    bal_sht = balance_sheet(COMPANY, MY_API_KEY)
    print(f"Bal Sheet {bal_sht}")
    cash_flw = cash_flow_statement(COMPANY, MY_API_KEY)
    print(f"Cash Flow {cash_flw}")
    ent_quote = enterprise_quote(COMPANY, MY_API_KEY)
    print(f"Ent Quote {ent_quote}")
    valuation_date = date.today()
    price = ent_quote[0]
    shares_outstanding = ent_quote[1]
    market_cap = ent_quote[2]
    fcff_data = calc_fcff(inc_stmnt, bal_sht, cash_flw)
    ebiat = fcff_data[0]
    capex = fcff_data[1]
    chng_nc_wc = fcff_data[2]
    depreciation = fcff_data[3]
    curr_yr_fcff = fcff_data[4]
   
    
    amort_schedule = capitalizerAndD(COMPANY, RD_YEARS, MY_API_KEY)
    print(f"Amortization Schedule {amort_schedule}")
    firm_reinvestment = calc_reinvestment(ebiat, capex, depreciation, chng_nc_wc, amort_schedule)
    adjusted_ebiat = calc_adj_ebiat(ebiat, amort_schedule)
    reinvestment_rate = firm_reinvestment / adjusted_ebiat
    print(f"Reinvestment rate; {reinvestment_rate}")
    try:
        valuation = Stock_Value(valuation_date , COMPANY, INDUSTRY, UNLEVERED_BETA, price, shares_outstanding, RISK_FREE, EQ_PREM)
        print(valuation)
    except Exception as e:
        print("An exception occured: ", e)
    
    eff_tax_rate = calc_tax_rate(inc_stmnt)
    return_on_capital = calc_return_on_capital(adjusted_ebiat, reinvestment_rate)
    growth_rate = calc_growth_rate(reinvestment_rate, return_on_capital)
    discount_rate = calc_discount_rate()
    pv_fcff = calc_pv_fcff(curr_yr_fcff,growth_rate, GROWTH_PERIOD )
    print(pv_fcff)
    
    print(valuation.valuation_date)
    print(valuation.ticker)
        
    print("DONE")
    
    

In [6]:
if __name__ == "__main__":
    main()

Inc Stmnt {'operating_income': [1282000000.0, -729000000.0, -241000000.0, 1650000000.0, 266000000.0], 'income_tax_expense': [403000000.0, 119000000.0, 506000000.0, 746000000.0, 200000000.0]}
Bal Sheet {'cash_and_equivalents': [1202000000.0, 1358000000.0, 1138000000.0, 1554000000.0, 2544000000.0], 'total_current_assets': [5207000000.0, 4881000000.0, 4946000000.0, 5598000000.0, 4891000000.0], 'current_long_debt': [75000000.0, 79000000.0, 1000000.0, 1000000.0], 'short_term_debt': [75000000.0, 79000000.0, 1000000.0, 1000000.0, 745000000.0], 'long_term_debt': [2573000000.0, 2469000000.0, 1806000000.0, 1727000000.0, 2214000000.0], 'total_current_liabilities': [3039000000.0, 3041000000.0, 2785000000.0, 3466000000.0, 3221000000.0], 'total_stockholders_equity': [5815000000.0, 3994000000.0, 4690000000.0, 4579000000.0, 3248000000.0]}
Cash Flow {'depreciation': [629000000.0, 760000000.0, 610000000.0, 267000000.0, 665000000.0], 'capex': [572000000.0, 549000000.0, 489000000.0, 389000000.0, 337000000